In [1]:
import cv2
import os
import random
import requests
import m3u8
import tempfile

class TwitchGrabber(object):
    def __init__(self, channel):
        self.channel = channel
        self.f = tempfile.NamedTemporaryFile(delete=False)

    def __enter__(self):
        return self

    def __exit__(self, *exc):
        if self.f:
          # os.unlink(self.f.name)
          self.f.close()

    def _get_auth_data(self):
        url = 'http://api.twitch.tv/api/channels/{channel}/access_token'.format(channel=self.channel)
        data = requests.get(url).json()
        return data

    def _get_livestream_m3u8_content(self):
        url = 'http://usher.twitch.tv/api/channel/hls/{channel}.m3u8'.format(channel=self.channel)
        data = self._get_auth_data()
        token, sig = data['token'], data['sig']

        params = dict(
            token=token,
            sig=sig,
            allow_audio_only=True,
            allow_source=True,
            allow_spectre=False,
            type='any',
            p=random.randint(0, 99999)
        )
        content = requests.get(url, params=params).text
        return content

    def get_highest_quality_stream(self):
        return m3u8.loads(self._get_livestream_m3u8_content()).playlists[0].uri

    def get_latest_segment_file_url(self):
        stream = m3u8.load(self.get_highest_quality_stream())
        obj = stream.segments[0]
        return obj.base_uri + obj.uri

    def get_latest_segment_file(self):
        # Write the data into our temp file
        response = requests.get(
            self.get_latest_segment_file_url(),
            stream=True
        )
        for block in response.iter_content(1024):
            self.f.write(block)
        return self.f


In [2]:
stream = TwitchGrabber('moonmoon_ow')

In [3]:
stream.f.name

'c:\\users\\alex\\appdata\\local\\temp\\tmp9h5xs7'

In [4]:
stream.get_latest_segment_file()

vidcap = cv2.VideoCapture(stream.f.name)

success, image = vidcap.read()
h, w, _ = image.shape
crop_h = h / 4
crop_w = w / 2

# image[y1:y2, x1:x2]
image = image[h - crop_h:h, crop_w:w]
# cv2.imwrite('debug.png', image)

hero = get_most_likely_heroes(image)[0]
if hero['score'] >= 9:
  print "Most likely: ", hero['hero']
else:
  print "Couldn't make out which character \033[1m{}\033[0m was playing with enough confidence.".format(
      stream.channel
  )


AttributeError: 'NoneType' object has no attribute 'shape'

In [5]:
success

False

In [6]:
##get_latest_segment_file step by step

In [7]:
f = open('assa.txt', 'w')
url = 'http://usher.twitch.tv/api/channel/hls/{channel}.m3u8'.format(channel=stream.channel)
data = stream._get_auth_data()
token, sig = data['token'], data['sig']
params = dict(
    token=token,
    sig=sig,
    allow_audio_only=True,
    allow_source=True,
    allow_spectre=False,
    type='any',
    p=random.randint(0, 99999)
)
content = requests.get(url, params=params).text
s = m3u8.load(m3u8.loads(content).playlists[0].uri)
obj = s.segments[0]

url_ls_video = obj.base_uri + obj.uri
response = requests.get(
    url_ls_video,
    stream=True
)
for block in response.iter_content(1024):
    f.write(block)
    

In [8]:
data

{u'mobile_restricted': False,
 u'sig': u'30f2473bcdc8739d61b9ba71f59345de96d77f7c',
 u'token': u'{"adblock":false,"player_type":null,"platform":null,"user_id":null,"channel":"moonmoon_ow","expires":1472150840,"chansub":{"view_until":1924905600,"restricted_bitrates":[]},"private":{"allowed_to_view":true},"privileged":false,"source_restricted":false,"https_required":false,"show_ads":true,"device_id":"43b3449f0b13b259"}'}

In [9]:
token

u'{"adblock":false,"player_type":null,"platform":null,"user_id":null,"channel":"moonmoon_ow","expires":1472150840,"chansub":{"view_until":1924905600,"restricted_bitrates":[]},"private":{"allowed_to_view":true},"privileged":false,"source_restricted":false,"https_required":false,"show_ads":true,"device_id":"43b3449f0b13b259"}'

In [10]:
sig

u'30f2473bcdc8739d61b9ba71f59345de96d77f7c'

In [11]:
params

{'allow_audio_only': True,
 'allow_source': True,
 'allow_spectre': False,
 'p': 26312,
 'sig': u'30f2473bcdc8739d61b9ba71f59345de96d77f7c',
 'token': u'{"adblock":false,"player_type":null,"platform":null,"user_id":null,"channel":"moonmoon_ow","expires":1472150840,"chansub":{"view_until":1924905600,"restricted_bitrates":[]},"private":{"allowed_to_view":true},"privileged":false,"source_restricted":false,"https_required":false,"show_ads":true,"device_id":"43b3449f0b13b259"}',
 'type': 'any'}

In [12]:
content

u'#EXTM3U\n#EXT-X-TWITCH-INFO:NODE="video16.lhr02",MANIFEST-NODE-TYPE="legacy",MANIFEST-NODE="video16.lhr02",SERVER-TIME="1472149641.51",USER-IP="88.21.131.122",SERVING-ID="fae6a8fee0b44c968695c8312e9dc1cf",CLUSTER="lhr02",ABS="false",BROADCAST-ID="22997872464",STREAM-TIME="11255.5137279",MANIFEST-CLUSTER="lhr02"\n#EXT-X-MEDIA:TYPE=VIDEO,GROUP-ID="chunked",NAME="Source",AUTOSELECT=YES,DEFAULT=YES\n#EXT-X-STREAM-INF:PROGRAM-ID=1,BANDWIDTH=4429000,RESOLUTION=1280x720,CODECS="avc1.640020,mp4a.40.2",VIDEO="chunked"\nhttp://video16.lhr02.hls.ttvnw.net/hls-a083b7/moonmoon_ow_22997872464_506128998/chunked/index-live.m3u8?token=id=5891381353884299120,bid=22997872464,exp=1472236041,node=video16.lhr02,nname=video16.lhr02,fmt=chunked&sig=ac6898a3c57febdedea0f8db082d90feafb0e286\n#EXT-X-MEDIA:TYPE=VIDEO,GROUP-ID="high",NAME="High",AUTOSELECT=YES,DEFAULT=YES\n#EXT-X-STREAM-INF:PROGRAM-ID=1,BANDWIDTH=1760000,RESOLUTION=1280x720,CODECS="avc1.66.31,mp4a.40.2",VIDEO="high"\nhttp://video16.lhr02.hls.ttv

In [13]:
s

In [14]:
obj

In [15]:
url_ls_video

u'http://video16.lhr02.hls.ttvnw.net/hls-a083b7/moonmoon_ow_22997872464_506128998/chunked/index-0000006075-TA0G.ts'

In [16]:
response

<Response [200]>

In [17]:
stream.get_latest_segment_file()
vidcap = cv2.VideoCapture(stream.f.name)
success, image = vidcap.read()
success

False

In [20]:
vidcap = cv2.VideoCapture(f.name)

success, image = vidcap.read()

In [21]:
success

False

In [22]:
f.close()